This notebook demonstrates most of the interactive DM functionality in `magpyx`

In [1]:
%matplotlib widget
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 100
plt.rcParams['image.origin'] = 'lower'

import numpy as np
from astropy.io import fits
from magpyx.dm import t2w_offload, connection_doctor, project_zernikes, eye_doctor
from magpyx import utils
import purepyindi as indi

def figure(figsize=None):
    'Temporary workaround for traditional figure behaviour with the ipympl widget backend'
    fig = plt.original_figure()
    if figsize:
        w, h =  figsize
    else:
        w, h = plt.rcParams['figure.figsize']
    fig.canvas.layout.height = str(h) + 'in'
    fig.canvas.layout.width = str(w) + 'in'
    return fig

try:
    plt.original_figure
except AttributeError:
    plt.original_figure = plt.figure
    plt.figure = figure

# 2K Connection Doctor

In [2]:
# mock up zrespM and respM_ref
zrespM = np.random.random((2500, 10, 10))
zrespM_ref = np.random.random((2500, 10, 10))

# use ALPAO zrespM to generalize to zrespMs of arbitrary size and no knowledge of DM shape... This might not be possible.

In [3]:
flagged_acts, flagged_connections = connection_doctor.check_actuator_functionality_2K(zrespM, zrespM_ref,
                                                                             actuator_mapfile='actuator_mapping_2019_09_27.csv',
                                                                             display=True,
                                                                             display_samtecs=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [4]:
connection_doctor.display_actuator_connections(filename='actuator_mapping_2019_09_27.csv')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(<Figure size 640x480 with 2 Axes>,
 <matplotlib.axes._subplots.AxesSubplot at 0x7f18e0faeef0>)

# ALPAO Connection Doctor

In [6]:
zrespM = 'zrespM_2019-03-20_23:43:03.fits'
zrespM_ref = np.random.random((121, 120, 120))
flagged_acts = connection_doctor.check_actuator_functionality_ALPAO(zrespM, zrespM_ref, display=True,)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Tweeter-Woofer Offload 

In [7]:
#tweeter_respM = np.random.random((2500, 120, 120))
#fits.writeto('zrespM_tweeter.fits', tweeter_respM)

In [8]:
wt_matrix, sing_vals, threshold = t2w_offload.get_offload_matrix('zrespM_2019-03-20_23:43:03.fits', 'zrespM_tweeter.fits',
                                          crosscheck=True, display=True, inverse=False, n_threshold=97)

INFO:offload:Thresholding after first 97 singular values.
INFO:offload:Mode #: tweeter RMS ---> woofer RMS (microns)
INFO:offload:Mode 0: 1.01 ---> 1157.95
INFO:offload:Mode 1: 1.01 ---> 1445.88
INFO:offload:Mode 2: 1.02 ---> 1131.19
INFO:offload:Mode 3: 1.04 ---> 1380.65
INFO:offload:Mode 4: 1.02 ---> 1479.57
INFO:offload:Mode 5: 1.03 ---> 1381.66
INFO:offload:Mode 6: 1.03 ---> 1098.58
INFO:offload:Mode 7: 1.04 ---> 1380.75
INFO:offload:Mode 8: 1.04 ---> 1568.80
INFO:offload:Mode 9: 1.04 ---> 1319.64
INFO:offload:Mode 10: 1.02 ---> 1056.60
INFO:offload:Mode 11: 1.06 ---> 1340.17
INFO:offload:Mode 12: 1.04 ---> 1338.19
INFO:offload:Mode 13: 1.06 ---> 1019.11
INFO:offload:Mode 14: 1.04 ---> 1266.33
INFO:offload:Mode 15: 1.04 ---> 1430.07
INFO:offload:Mode 16: 1.04 ---> 1296.00
INFO:offload:Mode 17: 1.04 ---> 1384.31
INFO:offload:Mode 18: 1.06 ---> 1534.05
INFO:offload:Mode 19: 1.06 ---> 1321.43
INFO:offload:Mode 20: 1.03 ---> 1397.17
INFO:offload:Mode 21: 1.05 ---> 1265.67
INFO:offload:

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Eye Doctor

In [11]:
client = indi.INDIClient('localhost', 7624)
client.start()

shmim = utils.ImageStream('test')

optimize a single mode

In [15]:
eye_doctor.eye_doctor(client, 'wooferModes', shmim, 3, [1,2,3], bounds=[-3, 3],
                      metric=eye_doctor.get_image_coresum, metric_dict={'radius': 10})

/opt/miniconda3/envs/py37/lib/python3.7/site-packages/scipy/ndimage/measurements.py:1359: RuntimeWarning: invalid value encountered in double_scalars
  for dir in range(input.ndim)]
/vagrant/vm/magpyx/magpyx/dm/eye_doctor.py:432: RuntimeWarning: invalid value encountered in double_scalars
  minima =  - b / (2 * c)
INFO:eye_doctor:Mode 1: Optimized mode coefficient from 1.0 to nan microns
INFO:eye_doctor:Mode 1: Optimized metric <function get_image_coresum at 0x7f18e2560730> from -0.0 to -0.0
INFO:eye_doctor:Mode 2: Optimized mode coefficient from 1.0 to nan microns
INFO:eye_doctor:Mode 2: Optimized metric <function get_image_coresum at 0x7f18e2560730> from -0.0 to -0.0
INFO:eye_doctor:Mode 3: Optimized mode coefficient from 1.0 to nan microns
INFO:eye_doctor:Mode 3: Optimized metric <function get_image_coresum at 0x7f18e2560730> from -0.0 to -0.0


optimize a custom range of modes

In [13]:
# get sequence working
seq_args = eye_doctor.build_sequence(client, 'wooferModes', shmim, 3, modes=[1,2,3],
                                     metric=eye_doctor.get_image_coresum, metric_dict={'radius' : 10})
for args in seq_args:
    eye_doctor.eye_doctor(*args)

INFO:eye_doctor:Mode 3: Optimized mode coefficient from 1.0 to nan microns
INFO:eye_doctor:Mode 3: Optimized metric <function get_image_coresum at 0x7f18e2560730> from -0.0 to -0.0
INFO:eye_doctor:Mode 1: Optimized mode coefficient from 1.0 to nan microns
INFO:eye_doctor:Mode 1: Optimized metric <function get_image_coresum at 0x7f18e2560730> from -0.0 to -0.0
INFO:eye_doctor:Mode 2: Optimized mode coefficient from 1.0 to nan microns
INFO:eye_doctor:Mode 2: Optimized metric <function get_image_coresum at 0x7f18e2560730> from -0.0 to -0.0
INFO:eye_doctor:Mode 3: Optimized mode coefficient from 1.0 to nan microns
INFO:eye_doctor:Mode 3: Optimized metric <function get_image_coresum at 0x7f18e2560730> from -0.0 to -0.0
INFO:eye_doctor:Mode 1: Optimized mode coefficient from 1.0 to nan microns
INFO:eye_doctor:Mode 1: Optimized metric <function get_image_coresum at 0x7f18e2560730> from -0.0 to -0.0
INFO:eye_doctor:Mode 2: Optimized mode coefficient from 1.0 to nan microns
INFO:eye_doctor:Mode

perform a "comprehensive" optimization

In [14]:
eye_doctor.eye_doctor_comprehensive(client, 'wooferModes', shmim, 3, modes=[3,4,5,2],
                                     metric=eye_doctor.get_image_coresum, metric_dict={'radius' : 10})

INFO:eye_doctor:Optimizing focus first!
INFO:eye_doctor:Mode 2: Optimized mode coefficient from 1.0 to nan microns
INFO:eye_doctor:Mode 2: Optimized metric <function get_image_coresum at 0x7f18e2560730> from -0.0 to -0.0
INFO:eye_doctor:Mode 5: Optimized mode coefficient from 1.0 to nan microns
INFO:eye_doctor:Mode 5: Optimized metric <function get_image_coresum at 0x7f18e2560730> from -0.0 to -0.0
INFO:eye_doctor:Mode 3: Optimized mode coefficient from 1.0 to nan microns
INFO:eye_doctor:Mode 3: Optimized metric <function get_image_coresum at 0x7f18e2560730> from -0.0 to -0.0
INFO:eye_doctor:Mode 4: Optimized mode coefficient from 1.0 to nan microns
INFO:eye_doctor:Mode 4: Optimized metric <function get_image_coresum at 0x7f18e2560730> from -0.0 to -0.0
INFO:eye_doctor:Mode 2: Optimized mode coefficient from 1.0 to nan microns
INFO:eye_doctor:Mode 2: Optimized metric <function get_image_coresum at 0x7f18e2560730> from -0.0 to -0.0
INFO:eye_doctor:Mode 2: Optimized mode coefficient from

write out the optimized flat from the DM shared memory image

In [4]:
eye_doctor.write_new_flat('woofer', update_symlink=False)

RuntimeError: Could not open shared memory image "dm00disp00"!